In [92]:
import numpy as np
from sklearn.preprocessing import normalize
from scipy.constants import epsilon_0 as ε_0, c, pi as π, e, hbar as ℏ
eV = e
from scipy.special import lpmn, sph_jn

I = np.identity(3)

## Green's functions
All vectors are expected to be in cartesian coordinates. Arguments and return values are in form of arrays ("lists") to enable fast batch processing.
### Free space

In [23]:
# Transverse delta; invalit at r1 = r2
def δ_T(r1, r2):
    print("Not implemented")
    return

# Longitudinal delta; invalid at r1 = r2
def δ_L(r1, r2):
    print("Not implemented")
    return

def G_fs(r1, r2, ω):
    r12 = np.linalg.norm((r1-r2), axis=1)
    g = np.exp(1j * r12 * ω/c) / (4*π*r12)
    return np.outer(g, I)    

def K_fs(r1, r2, ω):
    print("Not implemented")
    return

### Spherical particle (Mie solution)
TODO

In [33]:
# r[:,0] = x = r sin θ cos φ
# r[:,1] = y = r sin θ sin φ
# r[:,2] = z = r cos θ
def spherical_vector_waves(nmax, k, r):
    # k is array of scalars, r is array of vectors
    ρ = np.linalg.norm(r, axis=-1)
    r̂ = r/ρ[:,np.newaxis]
    cosθ = r̂[:,2]
    sinθ = np.sqrt(1-cosθ*cosθ)
    cosφ = r̂[:,0]/sinθ
    sinφ = r̂[:,1]/sinθ
    φ = np.arctan2(sinφ/cosφ)
    θ̂ = np.concatenate((cosθ*cosφ,cosθ*sinφ,-sinθ),axis=-1)
    φ̂ = np.concatenate((-sinφ, cosφ, 0), axis=-1)
    jnkr2 = sph_jn(nmax, k*r)
    # scipy.special.lpmn is for real arguments (see also clpmn)
    # + derivatives
    # m's are only positive, cf. Bohren&Huffman (4.23)
    Pmn2 = lpmn(nmax, nmax, cosθ)
    Memn=-(m/sinθ) * jnkr2[0][np.newaxis,:] * Pmn2[0] * sin
    
    
    #sph_jn(5,0.5)[0][np.newaxis,:]+1j*lpmn(5,5,0.01)[0]
    
    
    

## T-matrix

In [ ]:


def M(G, ω, R, μ): # as in PRA 70, 053823, eq. (37)
    #M = eye(N)
    
    

# Examples

# Playground

In [19]:
r = np.array([[1,0,0],[2,2,2]])
R = np.array([[0,1,0],[0,1,4]])
np.linalg.norm((r-R), axis=1)
G_fs(r, R, 2e8)

array([[ 0.03302973+0.04555572j,  0.00000000+0.j        ,
         0.00000000+0.j        ,  0.00000000+0.j        ,
         0.03302973+0.04555572j,  0.00000000+0.j        ,
         0.00000000+0.j        ,  0.00000000+0.j        ,
         0.03302973+0.04555572j],
       [-0.01107202+0.02410456j, -0.00000000+0.j        ,
        -0.00000000+0.j        , -0.00000000+0.j        ,
        -0.01107202+0.02410456j, -0.00000000+0.j        ,
        -0.00000000+0.j        , -0.00000000+0.j        ,
        -0.01107202+0.02410456j]])

In [25]:
r = np.array([[1,2,3],[4,5,65]])
r[:,2]

array([ 3, 65])

In [ ]:
ρ = np.linalg.norm(r, axis=1)
cosθ = r[:,2]/ρ

In [ ]:
cosθ = r[:,2]/ρ

sinθ = np.sqrt(1-cosθ)
sinθ 

In [34]:
r̂ = normalize(r, axis=1)

/home/necadam1/.local/lib/python3.4/site-packages/sklearn/utils/validation.py:498: UserWarning: The normalize function assumes floating point values as input, got int64
  "got %s" % (estimator, X.dtype))
/home/necadam1/.local/lib/python3.4/site-packages/sklearn/utils/extmath.py:71: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
  np.sqrt(norms, norms)
/home/necadam1/.local/lib/python3.4/site-packages/sklearn/preprocessing/data.py:619: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
  X /= norms[:, np.newaxis]


In [35]:
r̂

array([[0, 0, 1],
       [0, 0, 1]])

In [36]:
r

array([[ 1,  2,  3],
       [ 4,  5, 65]])

In [38]:
 normalize(r, axis=0)

/home/necadam1/.local/lib/python3.4/site-packages/sklearn/utils/extmath.py:71: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
  np.sqrt(norms, norms)
/home/necadam1/.local/lib/python3.4/site-packages/sklearn/preprocessing/data.py:619: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
  X /= norms[:, np.newaxis]


array([[0, 0, 0],
       [1, 1, 1]])

In [41]:
r̂ = r/ρ[:,np.newaxis]

In [42]:
r̂ 

array([[ 0.26726124,  0.53452248,  0.80178373],
       [ 0.06124203,  0.07655254,  0.99518296]])

In [53]:
lpmv(0, np.array([-1,0,1]),np.array([0.2,0.5,0.1]))

array([ 1. ,  1. ,  0.1])

In [61]:
lpmn(4,5,0.3)

(array([[  1.00000000e+00,   3.00000000e-01,  -3.65000000e-01,
          -3.82500000e-01,   7.29375000e-02,   3.45386250e-01],
        [  0.00000000e+00,  -9.53939201e-01,  -8.58545281e-01,
           7.86999841e-01,   1.69562693e+00,   1.60798377e-01],
        [  0.00000000e+00,   0.00000000e+00,   2.73000000e+00,
           4.09500000e+00,  -2.52525000e+00,  -1.04627250e+01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          -1.30212701e+01,  -2.73446672e+01,   8.65914462e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   8.69505000e+01,   2.34766350e+02]]),
 array([[  0.00000000e+00,   1.00000000e+00,   9.00000000e-01,
          -8.25000000e-01,  -1.77750000e+00,  -1.68562500e-01],
        [  0.00000000e+00,   3.14485451e-01,  -2.57878070e+00,
          -4.55217690e+00,   2.08818339e+00,   1.09149055e+01],
        [  0.00000000e+00,   0.00000000e+00,  -1.80000000e+00,
           1.09500000e+01,   3.03300000e+01,  

In [69]:
lpmv(1,1,0.3)
import scipy.special as ss


In [71]:
ss.lpmv(3,4,0.1)

-10.342894408723325

In [72]:
lpmv(0, 40, 0.001)

0.1252678976534484

In [75]:
ss.legendre(0)

poly1d([ 1.])

In [81]:
a = ss.lpmn(5,5,0.001)

In [82]:
a[0].shape

(6, 6)

In [83]:
a[1].shape

(6, 6)

In [85]:
a

(array([[  1.00000000e+00,   1.00000000e-03,  -4.99998500e-01,
          -1.49999750e-03,   3.74996250e-01,   1.87499125e-03],
        [  0.00000000e+00,  -9.99999500e-01,  -2.99999850e-03,
           1.49999175e+00,   7.49997875e-03,  -1.87497281e+00],
        [  0.00000000e+00,   0.00000000e+00,   2.99999700e+00,
           1.49999850e-02,  -7.49994000e+00,  -5.24997900e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          -1.49999775e+01,  -1.04999843e-01,   5.24994488e+01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   1.04999790e+02,   9.44998110e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,  -9.44997638e+02]]),
 array([[  0.00000000e+00,   1.00000000e+00,   3.00000000e-03,
          -1.49999250e+00,  -7.49998250e-03,   1.87497375e+00],
        [  0.00000000e+00,   1.00000050e-03,  -2.99999550e+00,
          -1.64999857e-02,   7.49993625e+00,  

In [89]:
lpmn(5, 5, 0.01)

(array([[  1.00000000e+00,   1.00000000e-02,  -4.99850000e-01,
          -1.49975000e-02,   3.74625044e-01,   1.87412508e-02],
        [  0.00000000e+00,  -9.99949999e-01,  -2.99985000e-02,
           1.49917504e+00,   7.49787508e-02,  -1.87228177e+00],
        [  0.00000000e+00,   0.00000000e+00,   2.99970000e+00,
           1.49985000e-01,  -7.49400053e+00,  -5.24790016e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          -1.49977501e+01,  -1.04984250e+00,   5.24448823e+01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   1.04979001e+02,   9.44811009e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,  -9.44763768e+02]]),
 array([[  0.00000000e+00,   1.00000000e+00,   3.00000000e-02,
          -1.49925000e+00,  -7.49825000e-02,   1.87237539e+00],
        [  0.00000000e+00,   1.00005000e-02,  -2.99954998e+00,
          -1.64985749e-01,   7.49362539e+00,  

In [100]:
lpmn(4,5,0.01)[0]

array([[  1.00000000e+00,   1.00000000e-02,  -4.99850000e-01,
         -1.49975000e-02,   3.74625044e-01,   1.87412508e-02],
       [  0.00000000e+00,  -9.99949999e-01,  -2.99985000e-02,
          1.49917504e+00,   7.49787508e-02,  -1.87228177e+00],
       [  0.00000000e+00,   0.00000000e+00,   2.99970000e+00,
          1.49985000e-01,  -7.49400053e+00,  -5.24790016e-01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         -1.49977501e+01,  -1.04984250e+00,   5.24448823e+01],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.04979001e+02,   9.44811009e+00]])

In [99]:
sph_jn(5,0.5)[0][np.newaxis,:]

array([[  9.58851077e-01,   1.62537031e-01,   1.63711066e-02,
          1.17403544e-03,   6.53896062e-05,   2.97746688e-06]])

In [102]:
sph_jn(5,0.5)[0][np.newaxis,:]+1j*lpmn(5,5,0.01)[0]

ValueError: m must be <= n.

In [ ]:
prod()